# Introduction

The data set refers to clients of a wholesale distributor. It includes the annual spending in monetary units (m.u.) on diverse product categories. The data set is also available at https://archive.ics.uci.edu/ml/datasets/Wholesale+customers#

Description of variables are as follows:

FRESH: annual spending (m.u.) on fresh products (Continuous);

MILK: annual spending (m.u.) on milk products (Continuous);

GROCERY: annual spending (m.u.)on grocery products (Continuous);

FROZEN: annual spending (m.u.)on frozen products (Continuous);

DETERGENTS_PAPER: annual spending (m.u.) on detergents and paper products (Continuous);

DELICATESSEN: annual spending (m.u.)on and delicatessen products (Continuous);

CHANNEL: customers Channel - Horeca (Hotel/Restaurant/Cafe) or Retail channel (Nominal);

REGION: customers Region Lisnon, Oporto or Other (Nominal)

The dataset gives data about sales of 6 category of products across 3 regions via 2 channel.

In [3]:
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Wholesale").getOrCreate()
sc = spark.sparkContext

from pyspark.sql import Row
import pyspark.sql.functions as func
from pyspark.sql.types import *
from pyspark.sql.functions import desc

sqlContext = SQLContext(sc)

In [10]:
import pandas as pd
df=pd.read_csv("Wholesale customers data.csv")

In [7]:
#Read csv file as dataframe
sale_df = sqlContext.createDataFrame(df)

In [9]:
sale_df.show(5)

+-------+------+-----+----+-------+------+----------------+----------+
|Channel|Region|Fresh|Milk|Grocery|Frozen|Detergents_Paper|Delicassen|
+-------+------+-----+----+-------+------+----------------+----------+
|      2|     3|12669|9656|   7561|   214|            2674|      1338|
|      2|     3| 7057|9810|   9568|  1762|            3293|      1776|
|      2|     3| 6353|8808|   7684|  2405|            3516|      7844|
|      1|     3|13265|1196|   4221|  6404|             507|      1788|
|      2|     3|22615|5410|   7198|  3915|            1777|      5185|
+-------+------+-----+----+-------+------+----------------+----------+
only showing top 5 rows



In [11]:
sale_df.cache()

DataFrame[Channel: bigint, Region: bigint, Fresh: bigint, Milk: bigint, Grocery: bigint, Frozen: bigint, Detergents_Paper: bigint, Delicassen: bigint]

In [15]:
#See the schema of the DF.
sale_df.printSchema()

root
 |-- Channel: long (nullable = true)
 |-- Region: long (nullable = true)
 |-- Fresh: long (nullable = true)
 |-- Milk: long (nullable = true)
 |-- Grocery: long (nullable = true)
 |-- Frozen: long (nullable = true)
 |-- Detergents_Paper: long (nullable = true)
 |-- Delicassen: long (nullable = true)



In [22]:
#Use select to view a single column or a set of chosen columns.
sale_df.select("Fresh").show(5)
sale_df.select("Channel","Region").show(5)

+-----+
|Fresh|
+-----+
|12669|
| 7057|
| 6353|
|13265|
|22615|
+-----+
only showing top 5 rows

+-------+------+
|Channel|Region|
+-------+------+
|      2|     3|
|      2|     3|
|      2|     3|
|      1|     3|
|      2|     3|
+-------+------+
only showing top 5 rows



In [28]:
#Use filter to see records with fresh sales more than 50000 only.
sale_df.filter( sale_df["Fresh"]>50000 ).show()

+-------+------+------+-----+-------+------+----------------+----------+
|Channel|Region| Fresh| Milk|Grocery|Frozen|Detergents_Paper|Delicassen|
+-------+------+------+-----+-------+------+----------------+----------+
|      1|     3| 56159|  555|    902| 10002|             212|      2916|
|      1|     3| 56082| 3504|   8906| 18028|            1480|      2498|
|      1|     3| 76237| 3473|   7102| 16538|             778|       918|
|      1|     3|112151|29627|  18148| 16745|            4948|      8550|
|      1|     1| 56083| 4563|   2124|  6422|             730|      3321|
|      1|     1| 53205| 4959|   7336|  3012|             967|       818|
|      1|     3| 68951| 4411|  12609|  8692|             751|      2406|
+-------+------+------+-----+-------+------+----------------+----------+



In [33]:
#Create aggregates on channels and regions variables.
sale_df.groupby("Channel").count().show()
sale_df.groupby("Region").count().show()

+-------+-----+
|Channel|count|
+-------+-----+
|      1|  298|
|      2|  142|
+-------+-----+

+------+-----+
|Region|count|
+------+-----+
|     1|   77|
|     3|  316|
|     2|   47|
+------+-----+



In [36]:
#Use describe to see summary statistics on dataframe.
sale_df.describe().show()

+-------+-------------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+
|summary|            Channel|           Region|             Fresh|             Milk|          Grocery|           Frozen|  Detergents_Paper|        Delicassen|
+-------+-------------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+
|  count|                440|              440|               440|              440|              440|              440|               440|               440|
|   mean| 1.3227272727272728|2.543181818181818|12000.297727272728|5796.265909090909|7951.277272727273|3071.931818181818|2881.4931818181817|1524.8704545454545|
| stddev|0.46805156947911386|  0.7742724492301|12647.328865076892|7380.377174570843|9503.162828994344|4854.673332592367| 4767.854447904201| 2820.105937369397|
|    min|                  1|                1

In [37]:
#Change datatype of Channels to Strings.
sale_df.select(sale_df.Channel.cast("string").alias('Channels')).show(5)

+--------+
|Channels|
+--------+
|       2|
|       2|
|       2|
|       1|
|       2|
+--------+
only showing top 5 rows



In [39]:
#Perform rollups on channels and regions.
sale_df.rollup("Channel","Region").avg().orderBy("Channel", "Region").show()


+-------+------+------------------+-----------------+------------------+------------------+------------------+------------------+---------------------+------------------+
|Channel|Region|      avg(Channel)|      avg(Region)|        avg(Fresh)|         avg(Milk)|      avg(Grocery)|       avg(Frozen)|avg(Detergents_Paper)|   avg(Delicassen)|
+-------+------+------------------+-----------------+------------------+------------------+------------------+------------------+---------------------+------------------+
|   null|  null|1.3227272727272728|2.543181818181818|12000.297727272728| 5796.265909090909| 7951.277272727273| 3071.931818181818|   2881.4931818181817|1524.8704545454545|
|      1|  null|               1.0| 2.51006711409396|13475.560402684563|3451.7248322147652|3962.1375838926174| 3748.251677852349|    790.5604026845638| 1415.956375838926|
|      1|     1|               1.0|              1.0|12902.254237288136|3870.2033898305085| 4026.135593220339| 3127.322033898305|    950.52542372